## 八皇后问题
- 在8×8格的国际象棋上摆放八个皇后，使其不能互相攻击，即任意两个皇后都不能处于同一行、同一列或同一斜线上，问有多少种摆法。


In [28]:
import numpy as np

'''
[[1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
]
以上列表示棋盘，列表中元素为1即表示该位置已摆上棋子
'''

queen_list = [[1, 0, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 1]]


# 将多层嵌套列表平铺成一个列表
def get_base_list(list2):
    spread_list = []
    if not isinstance(list2,list):
        return None
    for i in list2:
        temp_list = get_base_list(i)
        if temp_list:
            for temp in temp_list:
                spread_list.append(temp)
        else:
            spread_list.append(i)
    return spread_list

#获取列表的左斜线的索引值
def get_diagonal_index(list2):
    diag_index_list = []
    for i in range(len(list2)):

        k = i
        sub_index = []
        for j in range(i):
            sub_index.append((i,j))
            i -=1
            j +=1
            sub_index.append((i,j))
        if set(sub_index):
            diag_index_list.append(set(sub_index))
        if k+1 == len(list2):
            s = 1
            while s+1 < len(list2):
                temp = len(list2) - 1
                sub_index = []
                for j in range(s,len(list2)):
                    sub_index.append((temp,j))
                    temp -= 1
                diag_index_list.append(sub_index)
                s +=1
    #加入主对角线位置索引
    sub_index = [(i,i) for i in range(len(list2))]
    diag_index_list.append(sub_index)
    return diag_index_list

#获取列表右上角斜线'\'位置索引
'''
[(0,1), (1,2), (2,3)]
[(0,2), (1,3)]
'''
def get_right_diag_index(queen_list):
    s=1

    diag_index_list = []
    while s+1 < len(queen_list):
        sub_index=[]
        for i in range(s,len(queen_list)):
            sub_index.append((i-s,i))
        diag_index_list.append(set(sub_index))
        s +=1
    return diag_index_list

#根据位置索引遍历列表检查有无重复元素 1
def check_valid_list(list2,diag_index_list): 
    valid_flag = True
    for each_index in diag_index_list:
        #检查每一条斜线是否有重复元素 1
        index_data = [str(list2[k[0]][k[1]]) for k in each_index] 
        index_data_str = ''.join(index_data)
        dup_one_len = index_data_str.count('1')
        if dup_one_len > 1:
            valid_flag = False
            return 0
    return 1

#递归生成全列表组合即8** 8种,每返回一种组合进行合法性检查，最终返回满足条件的组合
def gen_all_list(r_list,length):
    all_list = []
    r_list_len = length
    if r_list_len == 1:
        for i in r_list:
            all_list.append(i)
        return all_list
    
    #嵌套长度-1，逐层生成列表
    r_list_len -= 1
    for i in range(len(r_list)):
        remain_all_list = gen_all_list(r_list,r_list_len)
        for rem_list in remain_all_list:
            sub_list =[]
            sub_list.append(r_list[i])
            sub_list.append(rem_list)
            #获取平铺列表
            temp = get_base_list(sub_list)
            #若当前已返回最上层列表，进行合法性检查
            if len(temp) == len(queen_list) ** 2:
                #利用numpy 将列表分成方阵形式
                temp_array = np.array(temp).reshape(len(queen_list), len(queen_list))
                temp_array_t = temp_array.T
                #求行列式的值，若为1，则表示行和列没有重复1
                if np.linalg.det(temp_array) == 1:
                    #调用get_diagonal_index获取列表的左下'/'斜线的索引值
                    diag_index_list = get_diagonal_index(temp_array)
                    #调用get_right_diag_index获取列表的右上'\'斜线的索引值  
                    diag_right_index_list = get_right_diag_index(temp_array)

                    valid_ind_1 = check_valid_list(temp_array,diag_index_list)
#                     print(temp_array)
#                     print('valid_ind_1 = ', valid_ind_1)                    
                    valid_ind_2 = check_valid_list(temp_array,diag_right_index_list)
#                     print('valid_ind_2 = ', valid_ind_2)
#                     检查转置方阵的斜线合法性
                    valid_ind_3 = check_valid_list(temp_array_t,diag_right_index_list)
#                     print('valid_ind_3 = ', valid_ind_3)
                    #如果所有检查均合法，即满足所有条件，将此列表添加到最终列表中
                    if valid_ind_1 and valid_ind_2 and valid_ind_3:
                        print('add valid list')
                        all_list.append(temp_array)

            else:
                all_list.append(sub_list)                    
    return all_list


            

In [29]:
total_list = gen_all_list(queen_list,len(queen_list))

add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list
add valid list


In [31]:

print("满足条件有%d 种组合", len(total_list))
for i in total_list:
    print(i)

满足条件有%d 种组合 44
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]]
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]]
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]]
[[0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]]
[[0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]]
[[0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 